In [32]:
import json
import numpy as np
import operator
import os
import pandas as pd

from functools import reduce
from pathlib import Path
from tqdm.auto import tqdm
from typing import Any, Dict, Mapping, Optional, Sequence

import keypoint_moseq as kpms

In [89]:
unsupervised_aging_dir = Path(os.environ["UNSUPERVISED_AGING"])

# project_name  = "2025-07-03_kpms-v2"
# model_name    = "2025-07-07_model-2"
project_name  = "2025-07-16_kpms-v3"
model_name    = "2025-07-16_model-4"
kpms_dir      = unsupervised_aging_dir / "data/kpms_projects"
# dataset_dir   = unsupervised_aging_dir / "data/datasets/combined_1126/"
# dataset_dir   = unsupervised_aging_dir / "data/datasets/combined_1126/"
# dataset_dir   = unsupervised_aging_dir / "data/datasets/c_1204/"
# poses_csv_dir = dataset_dir / "poses_csv"

# supervised_features_path = unsupervised_aging_dir / "data/archive/B6DO_video.csv"
# adj_metadata_path = unsupervised_aging_dir / "data/adj_metadata_sheets/combined_1126_adj_metadata.csv"

project_dir = kpms_dir / project_name

In [103]:
import shutil

result_to_use = 2
shutil.copy(project_dir / model_name / f"results-{result_to_use}.h5", project_dir / model_name / "results.h5")

PosixPath('/projects/kumar-lab/miaod/projects/unsupervised-aging/data/kpms_projects/2025-07-16_kpms-v3/2025-07-16_model-4/results.h5')

In [104]:
results = kpms.load_results(project_dir, model_name)

In [105]:
len(results)

492

In [106]:
sequences = [pose_dict["syllable"] for pose_dict in results.values()]
unique_syllables = sorted({s for seq in sequences for s in seq})
print(len(unique_syllables), max(unique_syllables))

96 97


In [107]:
# paths = """
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/LL5-B2B/2019-12-23_SPD/LL5-4_AgedB6-0430_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2021-01-18/DO-40-2141_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2020-10-27/DO-40-2181_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2020-07-28/DO-2D-4184_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2020-07-28/DO-20-1187_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2020-01-02/DO-20-1098_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2020-01-02/DO-20-1097_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2020-01-02/DO-20-1100_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2020-01-02/DO-40-2107_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2021-02-04/DO-40-2184_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV15-B7B8/2021-02-04/DO-40-2181_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/NV14-B7B8/2021-01-18/DO-20-1136_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/LL4-B2B/2019-09-04_SPD/LL4-4_AgedB6-0256_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/LL4-B2B/2019-09-04_SPD/LL4-3_AgedB6-0255_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/LL5-B6/2021-06-02_SPD/AgedB6-2056_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/LL5-B6/2021-06-02_SPD/AgedB6-1427_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/LL2-B6/2021-06-09_SPD/AgedB6-2259_trimmed_filtered_pose_est_v6.h5
# /projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/LL6-B2B/2019-12-23_SPD/LL6-2_AgedB6-0432_trimmed_filtered_pose_est_v6.h5
# """.split()
# paths = [s.removeprefix("/projects/kumar-lab/sabnig/Projects/UnsupervisedAging/nflow/results/")
#           .removesuffix("_trimmed_filtered_pose_est_v6.h5")
#           .replace("/", "__")
#          for s in paths]
# print(paths[:3])

In [108]:
# print("paths not found:")

# _c = 0
# for path in paths:
#     if f"{path}.csv" not in results.keys():
#         _c += 1
#         print(path)
# print(_c, len(paths))

In [109]:
# results = {f"{path}.csv": results[f"{path}.csv"] for path in paths if f"{path}.csv" in results}
# len(results)

In [110]:
def _get_latent_embedding_statistics() -> Dict[str, Sequence]:
    stats = []
    for _, info in tqdm(results.items()):
        latent_embeddings = info["latent_state"]

        means   = latent_embeddings.mean(axis=0)
        medians = np.median(latent_embeddings, axis=0)
        stds    = latent_embeddings.std(axis=0, ddof=0)

        features = np.concatenate((means, medians, stds))
        stats.append(features)

    trans = list(map(list, zip(*stats)))
    feature_len = len(trans)
    assert feature_len % 3 == 0

    ret = {}
    for i in range(feature_len):
        label = ("mean" if i < feature_len // 3 else
                 "median" if i < 2 * feature_len // 3 else "std")
        ret[f"latent_embedding_{label}_{i % (feature_len // 3)}"] = trans[i]
    return ret

latent_embedding_statistics = _get_latent_embedding_statistics()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 492/492 [00:09<00:00, 49.57it/s]


In [111]:
def _get_syllable_frequency_statistics(th: float = 0.0) -> Dict[str, Sequence[float]]:
    sequences = [info["syllable"] for info in results.values()]
    uniq = sorted({s for seq in sequences for s in seq})
    if th > 0.0:
        global_counts = {s: 0 for s in uniq}
        for seq in sequences:
            for s in seq:
                global_counts[s] += 1
        total = sum(global_counts.values())
        uniq = [s for s in uniq if total and global_counts[s] / total >= th]

    if not uniq:
        return {}

    idx = {s: i for i, s in enumerate(uniq)}
    n = len(uniq)

    out = {}
    for s in uniq:
        out[f"avg_bout_length_{s}"] = []
        out[f"total_duration_{s}"] = []
        out[f"num_bouts_{s}"] = []

    for _, info in tqdm(results.items()):
        seq = info["syllable"]
        dur = None
        for k in ("durations", "duration", "syllable_durations", "syllable_duration"):
            if k in info and hasattr(info[k], "__len__") and len(info[k]) == len(seq):
                dur = info[k]
                break

        total_len = [0]*n
        bout_cnt = [0]*n
        sum_dur = [0.0]*n

        prev = None
        run_len = 0
        for i, s in enumerate(seq):
            if s == prev:
                run_len += 1
            else:
                if prev in idx:
                    j = idx[prev]
                    total_len[j] += run_len
                    bout_cnt[j] += 1
                prev = s
                run_len = 1
            if s in idx and dur is not None:
                sum_dur[idx[s]] += float(dur[i])

        if prev in idx:
            j = idx[prev]
            total_len[j] += run_len
            bout_cnt[j] += 1

        if dur is None:
            for j in range(n):
                sum_dur[j] = float(total_len[j])

        for j, s in enumerate(uniq):
            abl = (total_len[j] / bout_cnt[j]) if bout_cnt[j] else 0.0
            out[f"avg_bout_length_{s}"].append(abl)
            out[f"total_duration_{s}"].append(sum_dur[j])
            out[f"num_bouts_{s}"].append(int(bout_cnt[j]))

    return out

syllable_frequency_statistics = _get_syllable_frequency_statistics()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 492/492 [00:08<00:00, 60.02it/s]


In [112]:
# def _old_get_syllable_frequency_statistics(th: float = 0.0) -> Dict[str, Sequence[int]]:
#     sequences = [info["syllable"] for info in results.values()]
#     uniq = sorted({s for seq in sequences for s in seq})
#     if th > 0.0:
#         global_counts = {s: 0 for s in uniq}
#         for seq in sequences:
#             for s in seq:
#                 global_counts[s] += 1
#         total = sum(global_counts.values())
#         uniq = [s for s in uniq if global_counts[s] / total >= th]

#     idx = {s: i for i, s in enumerate(uniq)}
#     n = len(uniq)

#     freqs_per_video = []
#     for _, info in tqdm(results.items()):
#         seq = info["syllable"]
#         cnt = np.zeros(n, dtype=int)
#         for s in seq:
#             if s in idx:
#                 cnt[idx[s]] += 1
#         total_tokens = len(seq)
#         freqs = cnt / total_tokens if total_tokens else cnt
#         freqs_per_video.append(freqs)

#     transposed = list(map(list, zip(*freqs_per_video)))
#     return {f"syllable_frequency_{s}": transposed[i] for i, s in enumerate(uniq)}

# old_syllable_frequency_statistics = _old_get_syllable_frequency_statistics()

In [113]:
def _get_metasyllable_transition_matrix(
    grouped_syllables: Optional[Mapping[str, Sequence[int]]] = None,
    *,
    ignore_unknown: bool = False,
    include_frequencies: bool = True,
) -> Dict[str, Sequence[float]]:
    if grouped_syllables is None:
        grouped_syllables = {}

    sequences = [info["syllable"] for info in results.values()]
    vocab_size = max(s for seq in sequences for s in seq) + 1
    all_indices = set(range(vocab_size))

    seen = set()
    for name, idxs in grouped_syllables.items():
        bad = set(idxs) - all_indices
        if bad:
            raise ValueError(f"Group '{name}' contains invalid indices {sorted(bad)}.")
        if seen.intersection(idxs):
            raise ValueError("Duplicate indices detected across groups.")
        seen.update(idxs)

    if not ignore_unknown:
        unknown = sorted(all_indices - seen)
        if unknown:
            grouped_syllables = dict(grouped_syllables)
            grouped_syllables["unknown"] = unknown

    names      = list(grouped_syllables.keys())
    idx_sets   = [set(grouped_syllables[n]) for n in names]
    g          = len(names)
    feats      = {f"transition_matrix_{a}_{b}": [] for a in names for b in names if a != b}
    if include_frequencies:
        feats.update({f"metasyllable_frequency_{n}": [] for n in names})

    idx_to_group = {}
    for gi, s in enumerate(idx_sets):
        for idx in s:
            idx_to_group[idx] = gi

    for _, info in tqdm(results.items()):
        seq = info["syllable"]
        G = np.zeros((g, g), dtype=float)
        for a, b in zip(seq[:-1], seq[1:]):
            if a in idx_to_group and b in idx_to_group:
                G[idx_to_group[a], idx_to_group[b]] += 1

        np.fill_diagonal(G, 0)
        row_sums = G.sum(axis=1, keepdims=True)
        np.divide(G, row_sums, out=G, where=row_sums != 0)

        for i, ai in enumerate(names):
            for j, bj in enumerate(names):
                if ai != bj:
                    feats[f"transition_matrix_{ai}_{bj}"].append(G[i, j])

        if include_frequencies:
            counts = np.zeros(g, dtype=int)
            for s in seq:
                if s in idx_to_group:
                    counts[idx_to_group[s]] += 1
            total_tokens = len(seq)
            freqs = counts / total_tokens if total_tokens else counts
            for i, name in enumerate(names):
                feats[f"metasyllable_frequency_{name}"].append(freqs[i])
    return feats


# nature-aging_634
# _metasyllable_groupings = {
#     "kpms_dendrogram_0": [0, 2, 10, 54, 35, 9, 30, 16, 26, 20, 6, 15],
#     "kpms_dendrogram_1": [24, 42, 52, 50, 48, 57, 33, 38, 60, 12, 58, 22, 43],
#     "kpms_dendrogram_2": [19, 59, 1, 3, 14, 18, 34, 5, 7, 46, 40, 4, 11, 45],
#     "kpms_dendrogram_3": [13, 8, 17, 39, 51, 21, 36, 61, 31, 49, 28, 44, 55, 37, 25, 32, 27, 56],
#     "kpms_dendrogram_4": [53, 62, 29, 41, 23, 47]
# }

# geroscience_492
_metasyllable_groupings = {
    "kpms_dendogram_0": [12, 20, 28, 14, 26],
    "kpms_dendogram_1": [33, 23, 39, 13, 3, 11, 18],
    "kpms_dendogram_2": [24, 9, 6, 25, 15, 21, 16, 35, 2, 10, 17],
    "kpms_dendogram_3": [4, 34, 22, 30, 27, 29, 32, 19],
    "kpms_dendogram_4": [5, 7, 8, 43, 55, 0, 1, 31],
}

# combined_1126
# _metasyllable_groupings = {
#     "kpms_dendrogram_0": [41, 11, 23, 39, 22, 37, 28, 32, 5, 34, 1, 31, 20, 13, 25],
#     "kpms_dendrogram_1": [45, 46],
#     "kpms_dendrogram_2": [44, 50, 4, 2, 18, 53, 24, 8, 35, 14, 15, 10, 17, 26, 30, 7, 43, 9, 42, 48, 6, 29],
#     "kpms_dendrogram_3": [47, 27, 36],
#     "kpms_dendrogram_4": [40, 21, 12, 33, 16, 0, 3, 19, 38],
# }

# c_1204
# _metasyllable_groupings = {
#     "kpms_dendrogram_0": [19, 30],
#     "kpms_dendrogram_1": [40, 34, 3, 5, 12, 31, 18, 22, 39, 2, 10],
#     "kpms_dendrogram_2": [38, 46, 29, 11, 42, 33, 7, 13, 0, 17, 1, 9, 24, 36, 26, 16, 32, 20, 8, 21],
#     "kpms_dendrogram_3": [27, 35],
#     "kpms_dendrogram_4": [23, 37, 25, 28, 15, 6, 4, 14]
# }

kpms_dendrogram_metasyllable_transition_matrix = _get_metasyllable_transition_matrix(_metasyllable_groupings, ignore_unknown=True, include_frequencies=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 492/492 [00:19<00:00, 25.64it/s]


In [114]:
def _merge_features(
    features: Sequence[Dict[str, Sequence[Any]]]
) -> pd.DataFrame:
    names = list(map(lambda path: path.removesuffix(".csv"), results.keys()))
    merged_features = reduce(operator.or_, [{"name": names}] + features, {})
    return pd.DataFrame(merged_features)

unsupervised_features_df = _merge_features([
    latent_embedding_statistics, 
    syllable_frequency_statistics,
    # old_syllable_frequency_statistics,
    kpms_dendrogram_metasyllable_transition_matrix
])
list(unsupervised_features_df.columns)

['name',
 'latent_embedding_mean_0',
 'latent_embedding_mean_1',
 'latent_embedding_mean_2',
 'latent_embedding_mean_3',
 'latent_embedding_mean_4',
 'latent_embedding_mean_5',
 'latent_embedding_mean_6',
 'latent_embedding_mean_7',
 'latent_embedding_mean_8',
 'latent_embedding_mean_9',
 'latent_embedding_mean_10',
 'latent_embedding_mean_11',
 'latent_embedding_median_0',
 'latent_embedding_median_1',
 'latent_embedding_median_2',
 'latent_embedding_median_3',
 'latent_embedding_median_4',
 'latent_embedding_median_5',
 'latent_embedding_median_6',
 'latent_embedding_median_7',
 'latent_embedding_median_8',
 'latent_embedding_median_9',
 'latent_embedding_median_10',
 'latent_embedding_median_11',
 'latent_embedding_std_0',
 'latent_embedding_std_1',
 'latent_embedding_std_2',
 'latent_embedding_std_3',
 'latent_embedding_std_4',
 'latent_embedding_std_5',
 'latent_embedding_std_6',
 'latent_embedding_std_7',
 'latent_embedding_std_8',
 'latent_embedding_std_9',
 'latent_embedding_st

In [115]:
unsupervised_features_df.to_csv(unsupervised_aging_dir / f"final_data_curation/2025-12-04_missing_do-embedding_{result_to_use}.csv")

In [36]:
metadata_df = pd.read_csv(unsupervised_aging_dir / "src" / "feature_extraction" / "2025-10-24_missing-do_supervised.csv")
features_df = metadata_df.merge(unsupervised_features_df, on="name", how="inner")
features_df

feature_matrix_output_path = unsupervised_aging_dir / "data/feature_matrices" / f"2025-10-28_c-new_78.csv"

features_df.to_csv(feature_matrix_output_path)

print(f"wrote feature matrix to  `{feature_matrix_output_path}`")

wrote feature matrix to  `/projects/kumar-lab/miaod/projects/unsupervised-aging/data/feature_matrices/2025-10-28_c-new_78.csv`


In [38]:
metadata_df

,name,mouse_id,sex,batch,tester,age,fi,weight,diet,strain,...,grooming_duration_secs,Rearing_supported_T5,Rearing_supported_T20,Rearing_supported_T55,Rearing_unsupported_T5,Rearing_unsupported_T20,Rearing_unsupported_T55,Grooming_T5,Grooming_T20,Grooming_T55
0,LL3-B6__2023-07-06_MFS__DO2279_DO_F_25083,DO-2279,Female,New,Sean Deats,25.285714,NaN,22.94,AL,DO,...,226.60000,0.032111,0.064167,0.079808,0.004556,0.002500,0.010212,0.002556,0.010667,0.033606
1,LL3-B6__2023-07-06_MFS__DO2343_DO_M_25058,DO-2343,Male,New,Sean Deats,25.285714,NaN,29.67,AL,DO,...,0.00000,0.148444,0.162389,0.112212,0.021667,0.026583,0.021717,0.000000,0.003972,0.134424
2,LL3-B6__2023-07-06_MFS__DO2342_DO_M_25057,DO-2342,Male,New,Sean Deats,25.285714,NaN,44.61,AL,DO,...,512.03333,0.028889,0.073000,0.090758,0.004556,0.010861,0.026283,0.030556,0.050583,0.047545
3,LL3-B6__2023-07-06_MFS__DO2281_DO_F_25085,DO-2281,Female,New,Sean Deats,25.285714,NaN,33.24,AL,DO,...,132.48385,0.054444,0.067389,0.071283,0.014333,0.045917,0.049768,0.013556,0.015556,0.045040
4,LL3-B6__2023-07-06_MFS__DO2282_DO_F_25086,DO-2282,Female,New,Sean Deats,25.285714,NaN,29.58,AL,DO,...,5.20690,0.032333,0.023111,0.025455,0.001222,0.001167,0.001576,0.012778,0.094944,0.061424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,LL3-B6%202023-07-10_MFS%20DO2324_DO_F_25128,DO-2324,Female,New,Sean Deats,25.857143,NaN,28.06,AL,DO,...,285.60001,0.018667,0.030611,0.028768,0.005000,0.034111,0.059970,0.023444,0.027389,0.048485
74,LL6-B6%202023-07-10_MFS%20DO2312_DO_F_25116,DO-2312,Female,New,Sean Deats,25.857143,NaN,30.19,AL,DO,...,0.00000,0.077667,0.105778,0.123273,0.003778,0.017611,0.015374,0.024778,0.049500,0.117071
75,LL3-B6%202023-07-06_MFS%20DO2340_DO_M_25056,DO-2340,Male,New,Sean Deats,25.285714,NaN,41.89,AL,DO,...,73.72414,0.146222,0.110472,0.066374,0.013667,0.036278,0.021394,0.015222,0.005611,0.121929
76,LL2-B6%202023-07-10_MFS%20DO2296_DO_F_25100,DO-2296,Female,New,Sean Deats,25.857143,NaN,30.91,AL,DO,...,51.51855,0.021111,0.024611,0.018232,0.001889,0.000861,0.000535,0.005111,0.008917,0.022677


In [17]:
### merge with metadata matrix (should have same number of rows)

metadata_df = pd.read_csv(adj_metadata_path)
metadata_unsupervised_features_df = metadata_df.merge(unsupervised_features_df, on="name", how="inner")
metadata_unsupervised_features_df

,name,mouse_id,sex,batch,tester,age,fi,weight,diet,strain,...,transition_matrix_kpms_dendrogram_2_kpms_dendrogram_3,transition_matrix_kpms_dendrogram_2_kpms_dendrogram_4,transition_matrix_kpms_dendrogram_3_kpms_dendrogram_0,transition_matrix_kpms_dendrogram_3_kpms_dendrogram_1,transition_matrix_kpms_dendrogram_3_kpms_dendrogram_2,transition_matrix_kpms_dendrogram_3_kpms_dendrogram_4,transition_matrix_kpms_dendrogram_4_kpms_dendrogram_0,transition_matrix_kpms_dendrogram_4_kpms_dendrogram_1,transition_matrix_kpms_dendrogram_4_kpms_dendrogram_2,transition_matrix_kpms_dendrogram_4_kpms_dendrogram_3
0,LL1-B2B__2019-12-24_SPD__LL1-1_AgedB6-0420,AgedB6-0420,Male,Batch1,Hannah,55.000000,7.287800,49.00,AL,B6,...,0.007951,0.648410,0.0,0.000000,0.212963,0.787037,0.0,0.091451,0.698807,0.209742
1,LL1-B2B__2020-01-02_SPD__LL1-1_AgedB6-0744,AgedB6-0744,Male,Batch1,Mackenzie,29.000000,4.320384,40.43,AL,B6,...,0.008258,0.507508,0.0,0.007143,0.271429,0.721429,0.0,0.065744,0.784314,0.149942
2,LL1-B2B__2020-01-02_SPD__LL1-4_AgedB6-0746,AgedB6-0746,Female,Batch1,Mackenzie,28.000000,2.320384,27.53,AL,B6,...,0.002203,0.673275,0.0,0.010152,0.203046,0.786802,0.0,0.063738,0.768303,0.167959
3,LL1-B2B__2020-06-16_SPD__AgedB6-0411,AgedB6-0411,Female,Batch1,Hannah,80.000000,5.787800,40.01,AL,B6,...,0.015407,0.515774,0.0,0.000000,0.365217,0.634783,0.0,0.118837,0.737042,0.144121
4,LL1-B2B__2020-06-17_SPD__AgedB6-0420,AgedB6-0420,Male,Batch1,Hannah,80.000000,8.787800,55.85,AL,B6,...,0.010949,0.633820,0.0,0.017699,0.159292,0.823009,0.0,0.147887,0.596244,0.255869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,Batch2__LL4-B6__2023-07-10_MFS__DO2305_DO_F_25109,DO-2305,Female,New,Sean Deats,25.857143,3.348214,45.21,AL,DO,...,0.007874,0.503937,0.0,0.014925,0.238806,0.746269,0.0,0.192308,0.665865,0.141827
1122,Batch2__LL5-B6__2023-07-06_MFS__DO2348_DO_M_25063,DO-2348,Male,New,Sean Deats,25.285714,2.570436,37.13,AL,DO,...,0.122347,0.380774,0.0,0.111849,0.258029,0.630122,0.0,0.130753,0.125523,0.743724
1123,Batch2__LL5-B6__2023-07-10_MFS__DO2307_DO_F_25111,DO-2307,Female,New,Sean Deats,25.857143,2.181547,40.38,AL,DO,...,0.008032,0.516734,0.0,0.000000,0.361345,0.638655,0.0,0.138169,0.666667,0.195164
1124,Batch2__LL1-B6__2023-07-06_MFS__DO2334_DO_M_25050,DO-2334,Male,New,Sean Deats,25.285714,2.959325,43.94,AL,DO,...,0.005814,0.434109,0.0,0.011765,0.317647,0.670588,0.0,0.274559,0.518892,0.206549


In [18]:
# metadata_df

In [19]:
supervised_features_df = pd.read_csv(supervised_features_path)
supervised_columns = [
    col for col in supervised_features_df.columns if col not in ["NetworkFilename", "PoseFilename", "Batch", "Tester", "AgeGroup", "MouseID", "Strain", "Diet", "Weight", "Sex", "AgeW", "AgeAtVid", "CFI_norm", "FLL", "score"]
]

# _filename_to_name = {
#     row["name"].split("__")[-1]: row["name"] for _, row in metadata_unsupervised_features_df.iterrows()
# }
# _names = []
# for _, row in supervised_features_df.iterrows():
#     if row["PoseFilename"].startswith("/"):
#         _names.append(_filename_to_name.get(row["PoseFilename"][1:], pd.NA))
#     else:
#         _names.append(row["NetworkFilename"].removesuffix(".avi").replace("/", "__"))
# supervised_features_df["name"] = _names
# supervised_features_df = supervised_features_df.drop_duplicates()

# ### drop only duplicated row (unused)
# _mask = (
#     (supervised_features_df["name"] == "LL3-B2B__2020-01-02_SPD__LL3-4_AgedB6-0842")
#     & (supervised_features_df["AgeW"] == 20)
# )
# supervised_features_df.drop(supervised_features_df[_mask].index, inplace=True)

# supervised_features_df = supervised_features_df[["name"] + supervised_columns].copy()

# supervised_features_df

In [20]:
old_feature_matrix = pd.read_csv("/projects/kumar-lab/miaod/projects/unsupervised-aging/data/feature_matrices/2025-07-23_feature-matrix__combined_1126__2025-07-20_kpms-v4_150__2025-07-20_model-1.csv")

trimmed_old_feature_matrix = old_feature_matrix[["name"] + supervised_columns]
trimmed_old_feature_matrix

,name,median_angular_velocity,median_base_tail_lateral_displacement,median_limb_duty_factor,median_nose_lateral_displacement,median_speed_cm_per_sec,median_step_length1,median_step_width,median_stride_length,median_tip_tail_lateral_displacement,...,grooming_duration_secs,Rearing_supported_T5,Rearing_supported_T20,Rearing_supported_T55,Rearing_unsupported_T5,Rearing_unsupported_T20,Rearing_unsupported_T55,Grooming_T5,Grooming_T20,Grooming_T55
0,LL1-B2B__2019-12-24_SPD__LL1-1_AgedB6-0420,0.902735,0.174143,0.583333,0.136155,13.372591,2.494440,3.142785,3.888184,0.463084,...,68.20147,0.047222,0.059278,0.045596,0.002000,0.002111,0.004485,0.018111,0.014250,0.024364
1,LL1-B2B__2020-01-02_SPD__LL1-1_AgedB6-0744,-3.972016,0.171122,0.590909,0.109959,19.498989,2.618690,2.832212,5.078335,0.471236,...,170.43687,0.088111,0.076472,0.070101,0.011222,0.057806,0.114293,0.022778,0.024194,0.049485
2,LL1-B2B__2020-01-02_SPD__LL1-4_AgedB6-0746,3.120141,0.195468,0.611111,0.128568,17.032001,2.849350,2.724150,4.977703,0.517820,...,148.81530,0.044444,0.048361,0.045515,0.008111,0.009500,0.015737,0.016556,0.017889,0.018061
3,LL1-B2B__2020-06-16_SPD__AgedB6-0411,-0.014360,0.218572,0.562500,0.125209,17.161134,2.724150,3.087758,4.991545,0.631964,...,69.73327,0.029667,0.018194,0.031596,0.001333,0.019694,0.024646,0.024778,0.016806,0.016273
4,LL1-B2B__2020-06-17_SPD__AgedB6-0420,1.817449,0.129974,0.600000,0.119354,10.664947,2.141280,3.206242,2.242566,0.341870,...,72.26663,0.017667,0.024333,0.028747,0.005333,0.002417,0.005384,0.012333,0.005833,0.017707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,Batch2__LL4-B6__2023-07-10_MFS__DO2305_DO_F_25109,-3.051629,0.120370,0.603896,0.078928,14.422121,3.547365,3.107371,6.821204,0.186639,...,65.20002,0.024556,0.020306,0.016343,0.000556,0.000417,0.000152,0.000000,0.014639,0.016939
1122,Batch2__LL5-B6__2023-07-06_MFS__DO2348_DO_M_25063,2.133048,0.162071,0.555556,0.092367,21.328195,2.723941,3.100602,6.138987,0.381408,...,125.46667,0.026333,0.032861,0.070576,0.000000,0.000000,0.001980,0.010778,0.027917,0.047323
1123,Batch2__LL5-B6__2023-07-10_MFS__DO2307_DO_F_25111,-11.242211,0.141575,0.583333,0.128946,16.562525,1.253099,2.968717,5.324403,0.485926,...,0.00000,0.091000,0.079083,0.064313,0.000000,0.004917,0.004071,0.023778,0.007806,0.108727
1124,Batch2__LL1-B6__2023-07-06_MFS__DO2334_DO_M_25050,-2.269023,0.150810,0.562500,0.094270,21.130466,3.073845,2.817691,5.330215,0.260465,...,116.33336,0.006333,0.006528,0.004667,0.006333,0.006222,0.004556,0.008111,0.020667,0.013303


In [21]:
features_df = metadata_unsupervised_features_df.merge(
    trimmed_old_feature_matrix,
    on="name",
    how="inner"
)
features_df

,name,mouse_id,sex,batch,tester,age,fi,weight,diet,strain,...,grooming_duration_secs,Rearing_supported_T5,Rearing_supported_T20,Rearing_supported_T55,Rearing_unsupported_T5,Rearing_unsupported_T20,Rearing_unsupported_T55,Grooming_T5,Grooming_T20,Grooming_T55
0,LL1-B2B__2019-12-24_SPD__LL1-1_AgedB6-0420,AgedB6-0420,Male,Batch1,Hannah,55.000000,7.287800,49.00,AL,B6,...,68.20147,0.047222,0.059278,0.045596,0.002000,0.002111,0.004485,0.018111,0.014250,0.024364
1,LL1-B2B__2020-01-02_SPD__LL1-1_AgedB6-0744,AgedB6-0744,Male,Batch1,Mackenzie,29.000000,4.320384,40.43,AL,B6,...,170.43687,0.088111,0.076472,0.070101,0.011222,0.057806,0.114293,0.022778,0.024194,0.049485
2,LL1-B2B__2020-01-02_SPD__LL1-4_AgedB6-0746,AgedB6-0746,Female,Batch1,Mackenzie,28.000000,2.320384,27.53,AL,B6,...,148.81530,0.044444,0.048361,0.045515,0.008111,0.009500,0.015737,0.016556,0.017889,0.018061
3,LL1-B2B__2020-06-16_SPD__AgedB6-0411,AgedB6-0411,Female,Batch1,Hannah,80.000000,5.787800,40.01,AL,B6,...,69.73327,0.029667,0.018194,0.031596,0.001333,0.019694,0.024646,0.024778,0.016806,0.016273
4,LL1-B2B__2020-06-17_SPD__AgedB6-0420,AgedB6-0420,Male,Batch1,Hannah,80.000000,8.787800,55.85,AL,B6,...,72.26663,0.017667,0.024333,0.028747,0.005333,0.002417,0.005384,0.012333,0.005833,0.017707
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,Batch2__LL4-B6__2023-07-10_MFS__DO2305_DO_F_25109,DO-2305,Female,New,Sean Deats,25.857143,3.348214,45.21,AL,DO,...,65.20002,0.024556,0.020306,0.016343,0.000556,0.000417,0.000152,0.000000,0.014639,0.016939
1122,Batch2__LL5-B6__2023-07-06_MFS__DO2348_DO_M_25063,DO-2348,Male,New,Sean Deats,25.285714,2.570436,37.13,AL,DO,...,125.46667,0.026333,0.032861,0.070576,0.000000,0.000000,0.001980,0.010778,0.027917,0.047323
1123,Batch2__LL5-B6__2023-07-10_MFS__DO2307_DO_F_25111,DO-2307,Female,New,Sean Deats,25.857143,2.181547,40.38,AL,DO,...,0.00000,0.091000,0.079083,0.064313,0.000000,0.004917,0.004071,0.023778,0.007806,0.108727
1124,Batch2__LL1-B6__2023-07-06_MFS__DO2334_DO_M_25050,DO-2334,Male,New,Sean Deats,25.285714,2.959325,43.94,AL,DO,...,116.33336,0.006333,0.006528,0.004667,0.006333,0.006222,0.004556,0.008111,0.020667,0.013303


In [22]:
# left_only = metadata_unsupervised_features_df[~metadata_unsupervised_features_df['name'].isin(supervised_features_df['name'])]
# right_only = supervised_features_df[~supervised_features_df['name'].isin(metadata_unsupervised_features_df['name'])]
# left_only

In [23]:
_all_unsupervised_columns = (
    list(latent_embedding_statistics.keys()) + 
    list(syllable_frequency_statistics.keys()) + 
    list(kpms_dendrogram_metasyllable_transition_matrix.keys())
)

# _old_all_unsupervised_columns = (
#     list(latent_embedding_statistics.keys()) + 
#     list(old_syllable_frequency_statistics.keys()) +
#     list(kpms_dendrogram_metasyllable_transition_matrix.keys())
# )

Xcats = {
    project_name: _all_unsupervised_columns,
    # f"{project_name}__old": _old_all_unsupervised_columns,
    "supervised": supervised_columns,
}
Xcats.keys()

dict_keys(['2025-09-20_kpms-v5_150_6', 'supervised'])

In [24]:
# features_df.drop("fll", axis=1, inplace=True)
rows_with_na = features_df[features_df.isna().any(axis=1)].copy()

rows_with_na["na_cols"] = (
    rows_with_na
    .isna()
    .apply(lambda r: [c for c, is_na in r.items() if is_na], axis=1)
)

rows_with_na["na_cols"]

0      [fll]
1      [fll]
2      [fll]
3      [fll]
4      [fll]
       ...  
629    [fll]
630    [fll]
631    [fll]
632    [fll]
633    [fll]
Name: na_cols, Length: 634, dtype: object

In [25]:
_uid = f"{dataset_dir.name}__{project_name}__{model_name}"

from datetime import datetime
current_datetime = datetime.now()
formatted_date = current_datetime.strftime("%Y-%m-%d")

feature_matrix_output_dir  = unsupervised_aging_dir / "data/feature_matrices"
# feature_matrix_output_path = feature_matrix_output_dir / f"{formatted_date}_feature-matrix__{_uid}.csv"
# xcats_output_path          = feature_matrix_output_dir / f"{formatted_date}_xcats__{_uid}.json"

feature_matrix_output_path = feature_matrix_output_dir / f"2025-10-28_c-old_1126.csv"
xcats_output_path          = feature_matrix_output_dir / f"2025-10-28_c_xcats.json"

features_df.to_csv(feature_matrix_output_path)
with xcats_output_path.open("w") as f:
    json.dump(Xcats, f, indent=2)

print(f"wrote feature matrix to  `{feature_matrix_output_path}`")
print(f"wrote X category JSON to `{xcats_output_path}`")
print(f"X category keys: {Xcats.keys()}")

wrote feature matrix to  `/projects/kumar-lab/miaod/projects/unsupervised-aging/data/feature_matrices/2025-10-28_c-old_1126.csv`
wrote X category JSON to `/projects/kumar-lab/miaod/projects/unsupervised-aging/data/feature_matrices/2025-10-28_c_xcats.json`
X category keys: dict_keys(['2025-09-20_kpms-v5_150_6', 'supervised'])
